# GROUP ASSIGNMENT - RMBA-2771 Fall 2025

Date: 23.09.2025 

Course: 2771 Research Methods for Business Analytics 

Instructor: Daniel Obermeier

Group: TXB B18

Henrik Peuker 72636

### Table of content

Select the hyperling to go to the respective section

- [1 Executive Summary](#1-executive-summary)
- [2 Introduction](#2-introduction)
- [3 Exploratory Analyis](#3-exploratory-analysis)
- [4 Method 1](#4-method-1)
- [5 Method 2](#5-method-2)
  - [5.1 Creating the model](#51-creating-the-model)
  - [5.2 Interpreting the outcome](#52-interpreting-the-outcome)
    - [5.2.1 Goodness-of-fit](#521-goodness-of-fit)
    - [5.2.2 Hypothesis testing](#522-hypothesis-testing)
    - [5.2.3 Odds ration](#523-odds-ratio)
- [6 Reflection on the use of AI](#6-reflection-on-the-use-of-ai)
- [7 Conclusion](#7-conclusion)


# 1. Executive Summary

# 2. Introduction

The data set "HR-Employee-Attrition" includes individual records of employees. Each row represents an individual employee, while the columns capture several demographic, educational, job-related and performance-related attributes. One of the main variables is Attrition, which shos if an employee has left the company. Attrition is recorded as a binary variable. 
The dataset includes variables such as age, gender, marital status, education level, job satisfaction, income. There is a mix of categorical, ordinal and numeric types which allows for a detailed analysis. 

When looking at the data set we have identified a novel, relevant and interesting research question regarding attrition  

The following research question serves as a guiding question for the analysis. To what extent do business travel frequency, distance from home, and income influence voluntary employee attrition?





To what extend do different job characteristics affect voluntary attrition? 

H1: 
Higher business travel frequency is positively associated with voluntary employee attrition.
Test: Chi-Square Tests or Logistic Regression

H2: Higher income is negatively associated with voluntary employee attrition?

H2: 
The relationship between business travel and voluntary attrition is stronger among married couples than singles.
Test: Chi-Square  or Logistic Regression


How do distance from home, income etc affect voluntary attrition? 


 


Research Question: Does promotion stagnation increase attrition, and is this effect affected by age?

H1: More years since last promotion are associated with higher attrition risk. 
H2: 
H3;:

# 3. Exploratory Analysis

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%pip install statsmodels
from statsmodels.stats.proportion import proportions_ztest
import statsmodels.formula.api as smf

Note: you may need to restart the kernel to use updated packages.


In [17]:
# Importing the data set and assigning it to the varible df 
df= pd.read_csv("HR-Employee-Attrition.csv")

In [18]:
# Creating a summary table to see the number of variable and the types to come up with a research question and get an overview of the data. 
summary_df = pd.DataFrame({
    "Variable": df.columns,
    "Type": [df[col].dtype.name for col in df.columns],
})

# Showing the summary, int64 are numerical values, whereas objects are categorical or binary values. 
print(summary_df)


                    Variable    Type
0                        Age   int64
1                  Attrition  object
2             BusinessTravel  object
3                  DailyRate   int64
4                 Department  object
5           DistanceFromHome   int64
6                  Education   int64
7             EducationField  object
8              EmployeeCount   int64
9             EmployeeNumber   int64
10   EnvironmentSatisfaction   int64
11                    Gender  object
12                HourlyRate   int64
13            JobInvolvement   int64
14                  JobLevel   int64
15                   JobRole  object
16           JobSatisfaction   int64
17             MaritalStatus  object
18             MonthlyIncome   int64
19               MonthlyRate   int64
20        NumCompaniesWorked   int64
21                    Over18  object
22                  OverTime  object
23         PercentSalaryHike   int64
24         PerformanceRating   int64
25  RelationshipSatisfaction   int64
2

In [19]:
pd.set_option('display.max_columns', None)
print(df.head(20))

    Age Attrition     BusinessTravel  DailyRate              Department  \
0    41       Yes      Travel_Rarely       1102                   Sales   
1    49        No  Travel_Frequently        279  Research & Development   
2    37       Yes      Travel_Rarely       1373  Research & Development   
3    33        No  Travel_Frequently       1392  Research & Development   
4    27        No      Travel_Rarely        591  Research & Development   
5    32        No  Travel_Frequently       1005  Research & Development   
6    59        No      Travel_Rarely       1324  Research & Development   
7    30        No      Travel_Rarely       1358  Research & Development   
8    38        No  Travel_Frequently        216  Research & Development   
9    36        No      Travel_Rarely       1299  Research & Development   
10   35        No      Travel_Rarely        809  Research & Development   
11   29        No      Travel_Rarely        153  Research & Development   
12   31        No      Tr

In [20]:
# Count unique values for the specified columns because we identified columns that are constant and therefore irrelevant for the analysis.
print("StandardHours value counts:")
print(df["StandardHours"].value_counts())

print("\nOver18 value counts:")
print(df["Over18"].value_counts())

print("\nEmployeeCount value counts:")
print(df["EmployeeCount"].value_counts())

# Drop the columns due to the described reasons.
df.drop(columns=["StandardHours", "Over18", "EmployeeCount"], inplace=True)


StandardHours value counts:
StandardHours
80    1470
Name: count, dtype: int64

Over18 value counts:
Over18
Y    1470
Name: count, dtype: int64

EmployeeCount value counts:
EmployeeCount
1    1470
Name: count, dtype: int64


In [21]:
# 3. Drop any exact duplicate rows
df.drop_duplicates(inplace=True)
print("After dropping duplicates:", df.shape)

# 4. Check for missing values
print("Missing per column:\n", df.isnull().sum())

# 5. Fill missing values
#    - For text (object) columns → “Unknown”
#    - For numeric columns → 0
for col in df.columns:
    if df[col].dtype == "object":
        df[col].fillna("Unknown", inplace=True)
    else:
        df[col].fillna(0, inplace=True)

# 6. Drop columns that won’t vary and aren’t informative
to_drop = ["EmployeeCount", "StandardHours", "Over18"]
for c in to_drop:
    if c in df.columns:
        df.drop(columns=c, inplace=True)

# 8. Re-inspect to confirm
print(df.head())
print(df.info())


After dropping duplicates: (1470, 32)
Missing per column:
 Age                         0
Attrition                   0
BusinessTravel              0
DailyRate                   0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeNumber              0
EnvironmentSatisfaction     0
Gender                      0
HourlyRate                  0
JobInvolvement              0
JobLevel                    0
JobRole                     0
JobSatisfaction             0
MaritalStatus               0
MonthlyIncome               0
MonthlyRate                 0
NumCompaniesWorked          0
OverTime                    0
PercentSalaryHike           0
PerformanceRating           0
RelationshipSatisfaction    0
StockOptionLevel            0
TotalWorkingYears           0
TrainingTimesLastYear       0
WorkLifeBalance             0
YearsAtCompany              0
YearsInCurrentRole          0
YearsSinceLastPromotion     0
YearsWithCu

# 4. Method 1 

In this code we do some basic exploratory analysis to get an overview of the data set. Firstly, we assess the data set to ensure we identified interesting variables. Next we checked for duplicates and missing values but could not find any. For text entries we used "Unknwon" and for numeric we used 0. Lastly, we have decided to drop columns that are not adding value in example "EmployeeCount", "StandardHours", "Over18". The entries in these columns are all the same and therefore do not add much value. 

In [22]:
# Importing the data set, I used trial and error for the encoding. 
df= pd.read_csv("HR-Employee-Attrition.csv", encoding="latin1")

# 1. Load the data
df = pd.read_csv("HR-Employee-Attrition.csv")

# 2. Peek at the top and structure
print(df.head())      # first 5 rows
print(df.info())      # column types & non-null counts

# 3. Drop any exact duplicate rows
df.drop_duplicates(inplace=True)
print("After dropping duplicates:", df.shape)

# 4. Check for missing values
print("Missing per column:\n", df.isnull().sum())

# 5. Fill missing values
#    - For text (object) columns → “Unknown”
#    - For numeric columns → 0
for col in df.columns:
    if df[col].dtype == "object":
        df[col].fillna("Unknown", inplace=True)
    else:
        df[col].fillna(0, inplace=True)

# 6. Drop columns that won’t vary and aren’t informative
to_drop = ["EmployeeCount", "StandardHours", "Over18"]
for c in to_drop:
    if c in df.columns:
        df.drop(columns=c, inplace=True)

# 8. Re-inspect to confirm
print(df.head())
print(df.info())



   Age Attrition     BusinessTravel  DailyRate              Department  \
0   41       Yes      Travel_Rarely       1102                   Sales   
1   49        No  Travel_Frequently        279  Research & Development   
2   37       Yes      Travel_Rarely       1373  Research & Development   
3   33        No  Travel_Frequently       1392  Research & Development   
4   27        No      Travel_Rarely        591  Research & Development   

   DistanceFromHome  Education EducationField  EmployeeCount  EmployeeNumber  \
0                 1          2  Life Sciences              1               1   
1                 8          1  Life Sciences              1               2   
2                 2          2          Other              1               4   
3                 3          4  Life Sciences              1               5   
4                 2          1        Medical              1               7   

   EnvironmentSatisfaction  Gender  HourlyRate  JobInvolvement  JobLevel  

In [23]:
# Simply checking the scale of the numeric columns 
cols_to_check = [
    'DistanceFromHome', 'Education', 'EnvironmentSatisfaction',
    'HourlyRate', 'JobInvolvement', 'JobLevel',
    'JobSatisfaction', 'RelationshipSatisfaction', 'WorkLifeBalance'
]

# Loop through each column and print min and max
for col in cols_to_check:
    min_val = df[col].min()
    max_val = df[col].max()
    print(f"{col}: Min = {min_val}, Max = {max_val}")


DistanceFromHome: Min = 1, Max = 29
Education: Min = 1, Max = 5
EnvironmentSatisfaction: Min = 1, Max = 4
HourlyRate: Min = 30, Max = 100
JobInvolvement: Min = 1, Max = 4
JobLevel: Min = 1, Max = 5
JobSatisfaction: Min = 1, Max = 4
RelationshipSatisfaction: Min = 1, Max = 4
WorkLifeBalance: Min = 1, Max = 4


In [24]:
# Detect outliers using the IQR method for numerical columns, this is not really necessary because our variable are nominal and therefore do not have outliers.
numeric_cols = df.select_dtypes(include='number').columns
outlier_summary = {}

for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    outlier_summary[col] = outliers.shape[0]

print("Outlier counts per column:")
print(outlier_summary)

# Check for irregularities in categorical columns
categorical_cols = df.select_dtypes(include='object').columns
for col in categorical_cols:
    print(f"\nUnique values in '{col}':")
    print(df[col].value_counts(dropna=False))

Outlier counts per column:
{'Age': 0, 'DailyRate': 0, 'DistanceFromHome': 0, 'Education': 0, 'EmployeeNumber': 0, 'EnvironmentSatisfaction': 0, 'HourlyRate': 0, 'JobInvolvement': 0, 'JobLevel': 0, 'JobSatisfaction': 0, 'MonthlyIncome': 114, 'MonthlyRate': 0, 'NumCompaniesWorked': 52, 'PercentSalaryHike': 0, 'PerformanceRating': 226, 'RelationshipSatisfaction': 0, 'StockOptionLevel': 85, 'TotalWorkingYears': 63, 'TrainingTimesLastYear': 238, 'WorkLifeBalance': 0, 'YearsAtCompany': 104, 'YearsInCurrentRole': 21, 'YearsSinceLastPromotion': 107, 'YearsWithCurrManager': 14}

Unique values in 'Attrition':
Attrition
No     1233
Yes     237
Name: count, dtype: int64

Unique values in 'BusinessTravel':
BusinessTravel
Travel_Rarely        1043
Travel_Frequently     277
Non-Travel            150
Name: count, dtype: int64

Unique values in 'Department':
Department
Research & Development    961
Sales                     446
Human Resources            63
Name: count, dtype: int64

Unique values in '

H1. There is no difference in levels of attrition among low-travelers and high travelers.

In [25]:

from scipy.stats import chi2_contingency

# 1. Build the contingency table
contingency_table = pd.crosstab(
    df['Attrition'],
    df['BusinessTravel']
)

# 2. Run the chi-square test
chi2, p, dof, expected = chi2_contingency(contingency_table)

# 3. Output the results
print("Contingency Table:")
print(contingency_table)

print(f"\nChi2 Statistic: {chi2:.4f}")
print(f"P-Value: {p:.4f}")
print(f"Degrees of Freedom: {dof}")

print("\nExpected Frequencies:")
print(pd.DataFrame(
    expected,
    index=contingency_table.index,
    columns=contingency_table.columns
))

Contingency Table:
BusinessTravel  Non-Travel  Travel_Frequently  Travel_Rarely
Attrition                                                   
No                     138                208            887
Yes                     12                 69            156

Chi2 Statistic: 24.1824
P-Value: 0.0000
Degrees of Freedom: 2

Expected Frequencies:
BusinessTravel  Non-Travel  Travel_Frequently  Travel_Rarely
Attrition                                                   
No              125.816327         232.340816     874.842857
Yes              24.183673          44.659184     168.157143


This implies that attrition and travel frequency are not unrelated. The contigency table shows that far fewer travelers choose to leave the company than if the observations were independent. And far more frequent travelers choose to leave the company. 

In [26]:
# 1. Define low- and high-travel groups
low_travel = df['BusinessTravel'].isin(['Non-Travel', 'Travel_Rarely'])
high_travel = df['BusinessTravel'] == 'Travel_Frequently'

# 2. Count attriters in each group
count_low  = df.loc[low_travel,  'Attrition'].eq('Yes').sum()
nobs_low   = low_travel.sum()

count_high = df.loc[high_travel, 'Attrition'].eq('Yes').sum()
nobs_high  = high_travel.sum()

# 3. Perform two-proportions z-test (one-sided: low < high)
stat, pval = proportions_ztest(
    count  = [count_low, count_high],
    nobs    = [nobs_low,   nobs_high],
    alternative = 'smaller'
)

print(f"Low-travel attrition rate:  {count_low}/{nobs_low:.0f} = {count_low/nobs_low:.3f}")
print(f"High-travel attrition rate: {count_high}/{nobs_high:.0f} = {count_high/nobs_high:.3f}")
print(f"\nZ-statistic = {stat:.3f},  one-sided p-value = {pval:.4f}")


Low-travel attrition rate:  168/1193 = 0.141
High-travel attrition rate: 69/277 = 0.249

Z-statistic = -4.415,  one-sided p-value = 0.0000


Looking at the low p-value there is significant evidence that the high travel attrition rate is significantly higher than the low travel attrition rate. 

# 5. Method 2 

## 5.1 Creating the model

In [30]:
print(summary_df)


                    Variable    Type
0                        Age   int64
1                  Attrition  object
2             BusinessTravel  object
3                  DailyRate   int64
4                 Department  object
5           DistanceFromHome   int64
6                  Education   int64
7             EducationField  object
8              EmployeeCount   int64
9             EmployeeNumber   int64
10   EnvironmentSatisfaction   int64
11                    Gender  object
12                HourlyRate   int64
13            JobInvolvement   int64
14                  JobLevel   int64
15                   JobRole  object
16           JobSatisfaction   int64
17             MaritalStatus  object
18             MonthlyIncome   int64
19               MonthlyRate   int64
20        NumCompaniesWorked   int64
21                    Over18  object
22                  OverTime  object
23         PercentSalaryHike   int64
24         PerformanceRating   int64
25  RelationshipSatisfaction   int64
2

In [35]:
# Encode Attrition as 0/1
df['attrit_bin'] = df['Attrition'].map({'No': 0, 'Yes': 1})

# Fit logistic regression

model = smf.logit('attrit_bin ~ BusinessTravel + MaritalStatus + MonthlyIncome + DistanceFromHome + WorkLifeBalance + PerformanceRating + JobSatisfaction + EnvironmentSatisfaction', data=df).fit()
print(model.summary())



Optimization terminated successfully.
         Current function value: 0.387370
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:             attrit_bin   No. Observations:                 1470
Model:                          Logit   Df Residuals:                     1459
Method:                           MLE   Df Model:                           10
Date:                Tue, 23 Sep 2025   Pseudo R-squ.:                  0.1230
Time:                        15:45:19   Log-Likelihood:                -569.43
converged:                       True   LL-Null:                       -649.29
Covariance Type:            nonrobust   LLR p-value:                 3.712e-29
                                          coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                              -0.2907      0.852 

## 5.2 Interpreting the outcome 


### 5.2.1 Goodness-of-fit

### 5.2.2 Hypothesis testing

### 5.2.3 Odds ratio

In [ ]:
mcfadden_r2 = 1 - (model.llf / model.llnull)
print(f"McFadden's pseudo R-squared: {mcfadden_r2}")

#Cox & Snell R-Squared
n = len(df)  # number of observations
cox_snell_r2 = 1 - np.exp((model.llnull - model.llf) * 2 / n)
print(f"Cox & Snell pseudo R-squared: {cox_snell_r2}")

# Nagelkerke R-Squared
nagelkerke_r2 = cox_snell_r2 / (1 - np.exp(-model.llnull * 2 / n))
print(f"Nagelkerke pseudo R-squared: {nagelkerke_r2}")

McFadden's pseudo R-squared: 0.05142256871498918
Cox & Snell pseudo R-squared: 0.04440984118616731
Nagelkerke pseudo R-squared: -0.03129468939990255


#### Interpretation

We can argue that the model is not very useful for predictions due to the the low R-squared values. It can still be used for hypothesis testing. 

# 6. Reflection on the use of AI

We primarily used AI to help us solving issues. Our main contribution was finding the problems that require a solution. (...)

# 7. Conclusion